In [112]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

%matplotlib inline

import io, os, glob, gc
import timeit
from nbformat import current

HISTOGRAM_BITS_PER_COLOR = 2

def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)

execute_notebook('cuav.ipynb')

ADAPTIVE_HISTOGRAM_THRESHOLD = False
HISTOGRAM_SUGGESTED_THRESHOLD = 30

SHOW_DUPES = True
#MIN_DEDUPE_REGIONS = 0
#DEDUPE_REGIONS = False
#REGION_MERGE_PIXEL_DISTANCE = 50
#MIN_REGION_SCORE = 0

In [113]:
def ProcessFile(image):
        bin = InitBin(image)
        lut = InitLUT(bin)
        qimage = quantize(image, lut)

        hist = BuildHistogram(qimage)
        mask = HistogramThreshold(qimage, hist)
        print 'Histogram Count Threshold: %i' % HISTOGRAM_COUNT_THRESHOLD

        regions = AssignRegions(mask)
        print 'Total regions after assignment: %i' % regions.num_regions
        ScoreRegions(regions, qimage, hist)
        print 'Total regions after scoring: %i' % regions.num_regions
        PruneRegions(regions)
        
        del bin,lut,qimage,hist,mask
        gc.collect()
        
        return regions

In [114]:
#from guppy import hpy
#hp = hpy()
#before = hp.heap()

## critical section here
#raw = cv2.imread('batch/few/1.png')
#image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
#regions = ProcessFile(image)

#after = hp.heap()
#leftover = after - before
##import pdb; pdb.set_trace()

#print leftover.byclass

In [115]:
#raw = cv2.imread('batch/od/image_car_full.png')
#image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)

#%timeit ProcessFile(image)
#%prun regions = ProcessFile(image)

In [116]:
#for dirname, dirnames, filenames in os.walk('batch/few/'):
#    for filename in sorted(filenames):
#        print(os.path.join(dirname, filename))


In [117]:
PrintSettings()

raw_files_dir = 'batch/od/'
proc_files_dir = raw_files_dir + 'results/'

if not os.path.exists(proc_files_dir):
    os.makedirs(proc_files_dir)
    
for f in sorted(glob.glob(raw_files_dir + '*.*')):
    file = f.replace(raw_files_dir,'')
    if file.lower().endswith('.jpg') or file.lower().endswith(('png')):
        print file
        raw = cv2.imread(f)
        image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
        
        regions = ProcessFile(image)
        
        pruned_image = np.copy(image)
        MarkRegions(pruned_image, regions)
        bgr = cv2.cvtColor(pruned_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(proc_files_dir + file, bgr)
        print '\n'
    else:
        print 'Skipping %s' % file

Adaptive Threshold Disabled with count set at 30 

HISTOGRAM_BITS_PER_COLOR = 2
NUM_BINS = 4
COLOR_SPECTRUM = 65
ADAPTIVE_HISTOGRAM_THRESHOLD = False
HISTOGRAM_CEILING_THRESHOLD = 1500
HISTOGRAM_FLOOR_THRESHOLD = 50
SHOW_DUPES = True
DEDUPE_REGIONS = True
MIN_DEDUPE_REGIONS = 5
REGION_MERGE_PIXEL_DISTANCE = 20
MIN_REGION_SCORE = 0

image_car_full.png
InitBin:
min_r=0, max_r=255, bin_spacing.r=64
min_g=0, max_g=255, bin_spacing.g=64
min_b=34, max_b=255, bin_spacing.b=56
image height=2048,width=2448,depth=3
Histogram Count Threshold: 39
Total regions after assignment: 37
Total regions after scoring: 37
Min Region Score: 25
Average Region Score: 33
Median Region Score: 0
Max Region Score: 788
Total Pruned Regions: 4
   Region Score  Region Pixel Avg  Mod  Div Region Active
0            25                61   25    0          True
1           788                21   11   37          True
2           400                41   31    9          True
3            25                61   25    0  

In [118]:
%sx ipython nbconvert "oddiagnostic.ipynb"

['[NbConvertApp] Converting notebook oddiagnostic.ipynb to html',
 '[NbConvertApp] Writing 213024 bytes to oddiagnostic.html']

In [119]:
%sx mv "oddiagnostic.html" $proc_files_dir

[]